In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os

os.environ['SPOTIPY_CLIENT_ID']='3e8627e438fa4781abf064e0d3bc7b6f'
os.environ['SPOTIPY_CLIENT_SECRET']='0ac3077f50b54e248fc32c64c8fc9e13'

# initialize spotipy
client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# read in data from playlist_name_cleaned.csv
# df = pd.read_csv('cleaned_playlist_uri.csv')

In [5]:
import fasttext 
from fasttext import train_supervised
from cleantext import clean
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import emoji, re

def clean_text(text):
    text = clean(text, no_line_breaks=True, no_urls=True, no_punct=True, no_digits=True, replace_with_punct="", replace_with_digit="")
    text = emoji.replace_emoji(text, replace='')
    text = text.strip()
    text = text.replace('~', '')
    text = text.replace('_', '')
    text = text.replace('$', '')
    text = text.replace('^', '')
    # remove all mathemtical symbols, including =, +, -, *, /, <, >, %
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text_tokens = word_tokenize(text)
    cleaned_tokens = [w for w in text_tokens if not w in stopwords.words('english')]
    text = (" ").join(cleaned_tokens)
    if len(text) < 3:
        return ""
    return text

print(len(df))
# rename playlist_names with cleaned text, remove empty rows
df['playlist_name'] = df['playlist_name'].apply(clean_text)
df = df[df['playlist_name'] != ""]
print(len(df))

# save to a new csv file
df.to_csv('cleaner_playlist_uri.csv', index=False)

# convert df to dictionary
playlist_dict = df.set_index('playlist_name').T.to_dict('list')

for key in list(playlist_dict.keys()):
    tracklist = eval(playlist_dict[key][0])
    playlist_dict[key] = tracklist

# place each key on a new line in a text file
with open('clean_playlist.txt', 'w') as f:
    for key in playlist_dict.keys():
        f.write(key + ' ')
        for track in playlist_dict[key]:
            f.write('__label__' + track + ' ')
        f.write('\n')

53733
53733


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_1485/3243427280.py:37: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  playlist_dict = df.set_index('playlist_name').T.to_dict('list')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [1]:
import random
# randomly split the data into 80% training and 20% testing
with open('clean_playlist.txt', 'r') as f:
    data = f.readlines()
    random.shuffle(data)
    train = data[:int(len(data)*0.8)]
    test = data[int(len(data)*0.8):]
    # write train.txt and test.txt
    with open('playlist.train', 'w') as f:
        for line in train:
            f.write(line)
    with open('playlist.test', 'w') as f:
        for line in test:
            f.write(line)

In [1]:
import fasttext 
from fasttext import train_supervised

# train fasttext model on the playlist names, using track uri as the label
model = fasttext.train_supervised(input='clean_playlist.txt', lr=0.5, epoch=500, wordNgrams=4, bucket=200000, dim=50, loss='hs')

Read 2M words
Number of words:  8818
Number of labels: 558710
Progress: 100.0% words/sec/thread: 1263230 lr:  0.000000 avg.loss: 11.029614 ETA:   0h 0m 0s 37.8% words/sec/thread: 1276024 lr:  0.311233 avg.loss: 13.775496 ETA:   0h 1m36s 52.0% words/sec/thread: 1233817 lr:  0.240097 avg.loss: 12.900927 ETA:   0h 1m17s 86.0% words/sec/thread: 1267403 lr:  0.070012 avg.loss: 11.465023 ETA:   0h 0m21s


In [9]:
df = pd.read_csv('cleaner_playlist_uri.csv')

In [ ]:
prediction_words = "sad depressing feels"
# predict the track uri's for the given words
track_predictions = model.predict(prediction_words, k=20)
# select the first 10 unique predicted track uri's
track_predictions = np.unique(track_predictions[0][:10], return_counts=True)

# get the track names for the predicted track uri's
for track in track_predictions[0]:
    print(spotify.track(track[9:])['name'])

# get a new dataframe containing only playlists with any of the words in the playlist name
rslt_df = df.loc[df['playlist_name'].str.contains(prediction_words.replace(' ', ' | ')) == True]
playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')

for key in list(playlist_dict.keys()):
    tracklist = eval(playlist_dict[key][0])
    playlist_dict[key] = tracklist

accuracy = []
# for each top 10 predicted track, add 1 to the list if it is in one of the playlists, else add 0
for track in track_predictions[0]:
    for key in playlist_dict.keys():
        if track[9:] in playlist_dict[key]:
            accuracy.append(1)
            print(spotify.track(track[9:])['name'] + " is in " + key)
            break

# calculate the accuracy
print(sum(accuracy)/10)

In [86]:
import random
# from the set of all playlist names, get the unique words
all_words = set()
for playlist in df['playlist_name']:
    for word in str(playlist).split():
        all_words.add(word)

# create a random list of 1000 sets of 1-5 words from the set of all words
random_descriptions = []
for i in range(1000):
    description_words = random.sample(all_words, random.randint(1, 5))
    description = ' '.join(description_words)
    random_descriptions.append(description)


1000


/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_6883/805594410.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  description_words = random.sample(all_words, random.randint(1, 5))


In [ ]:
#***** NEED TO ADD COMPARISON TO RANDOM PLAYLISTS *****

overall_accuracy = []
# # predict the track uri's for the given words
for description in random_descriptions:
    track_predictions = model.predict(description, k=20)
    track_predictions = np.unique(track_predictions[0][:10], return_counts=True)
    # get the track names for the predicted track uri's
    for track in track_predictions[0]:
        print(spotify.track(track[9:])['name'])
    # get a new dataframe containing only playlists with any of the words in the playlist name
    rslt_df = df.loc[df['playlist_name'].str.contains(description.replace(' ', ' | ')) == True]
    playlist_dict = rslt_df.set_index('playlist_name').T.to_dict('list')
    for key in list(playlist_dict.keys()):
        tracklist = eval(playlist_dict[key][0])
        playlist_dict[key] = tracklist
    accuracy = []
    # for each top 10 predicted track, add 1 to the list if it is in one of the playlists, else add 0
    print("Input Description: " + description + "\n")
    print("Reccomended Tracks: ")
    for track in track_predictions[0]:
        print(spotify.track(track[9:])['name'])
        for key in playlist_dict.keys():
            if track[9:] in playlist_dict[key]:
                accuracy.append(1)
                break
    # calculate the accuracy
    overall_accuracy.append(sum(accuracy)/10)
    # write to a text file
    with open('fasttext_results.txt', 'a') as f:
        f.write(str(sum(accuracy)/10) + '\n')

print(sum(overall_accuracy)/1000)

In [48]:
# introduce some randomness so it skips some of the closest values, so that it doesn't pick the 
# absolute closest songs every time
# start with semnatic textual similarity 
# two vectors, compare how close they are to each other, could also just do an embedding 
# google colab gpu machines, or spin off an aws account, and spin off a gpu machine from aws or azure
# if you have access to the spotify api features, do it both ways
# once with word embeddings, once with music features, then combine them, depending on how much you want to weight them 

# create some metrics yourself for evaluation, or look at how many in playlists vs. outof plylist songs are recommended 
# vs. random sampling 

# signal processing for how to extract the classical features 

# 1: Get a sentence embedder
# 2: Put embeddings together in a latent space
# 3: Use KNN to get closest pairs
# 1: Get music features
# 2: Put music features in latent space
# 3: use KNN to get closest pairs
# 1: Combine two methods for song recommendations

# collaborative filtering

In [65]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

data = []
for playlist in df['playlist_name']:
    data.append(str(playlist))

# create a list of tagged documents, where each document is a playlist name
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =0)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [85]:
model= Doc2Vec.load("d2v.model")

for test in random_descriptions:
    test_data = word_tokenize(test.lower())
    v1 = model.infer_vector(test_data)
    # given the vector of a document, find the most similar document in the training data
    similar_doc = model.docvecs.most_similar([v1], topn=50)
    print("Input Description: " + test + "\n")
    print("Most Similar Playlists: \n")
    printed = []
    # print the most similar playlists with no repeats
    for i in range(len(similar_doc)):
        name = data[int(similar_doc[i][0])]
        if name not in printed:
            print(data[int(similar_doc[i][0])])
            printed.append(name)
    print("\n")

/var/folders/v1/ljzyxbgj1xx_syzq56dgnsk00000gn/T/ipykernel_6883/4029271612.py:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar([v1], topn=50)


Input Description: gammelt

Most Similar Playlists: 

girl power
endless
power
feelgood
endless love


Input Description: resist noughties photograph dancebeats churrasco

Most Similar Playlists: 

n e w
w e
n e
jorge
n c e
u e r
r v e
v p r w v e
p e
jorge e mateus
r b
r n b
mmj
rockmetal
simplicity
rap
v b e
e h e c
c u n r


Input Description: joey heldens vicky jersey

Most Similar Playlists: 

joey
ariel camacho
ariel
joey bada
yabanc
mode
vicky
depeche mode
airplane mode
beast mode
krunk
exo
acoustic
lone
acoustic songs


Input Description: translation xxl coltrane ashleys

Most Similar Playlists: 

best playlist ever
best playlist
best song ever
loft
best
best ever
softball playlist
title
best songs ever
uke
best best
best song


Input Description: lemonade snazzy sampler living strokes

Most Similar Playlists: 

morning jams
latest hits
top hits
latest
hits
todays top hits
monday morning


Input Description: sadboys mias kizomba

Most Similar Playlists: 

kizomba
mel
sigh
fanny